# Анализ данных транзакций

В данном ноутбуке будет проведен анализ данных  с целью выявления фрода. Действуем по модели ETL:
- Extract (извлечение данных)
- Transform (Обработка и визуализация)
- Load (Выходные данные)

## Часть первая - Extract

In [2]:
# импортируем библиотеки
import pandas as pd # работа с данными
import numpy as np # работа с вычислениями
import seaborn as sns # визуализация
import json # парсинг json файлов

Распакуем json файл и поместим его в Pandas.DataFrame

In [3]:
# из-за структуры файла сразу в pandas запихнуть не получится, поэтому требуется первичная обработка
with open('transactions.json') as raw_json:
    content = json.load(raw_json)
    content1 = content['transactions']

content1

{'459270924': {'date': '2020-05-01T00:00:29',
  'card': '59649132026167121328',
  'account': '40817810000001139973',
  'account_valid_to': '2036-01-16T00:00:00',
  'client': '3-95179',
  'last_name': 'Мисик',
  'first_name': 'Сергей',
  'patronymic': 'Николаевич',
  'date_of_birth': '1938-06-25T00:00:00',
  'passport': 7076445954,
  'passport_valid_to': '2022-11-09T00:00:00',
  'phone': '+79497481039',
  'oper_type': 'Пополнение',
  'amount': 31576.6,
  'oper_result': 'Отказ',
  'terminal': 'POS43792',
  'terminal_type': 'POS',
  'city': 'Славянск-на-Кубани',
  'address': 'Славянск-на-Кубани, ул. Клецкая, д. 86'},
 '459270943': {'date': '2020-05-01T00:02:46',
  'card': '54236863445669263724',
  'account': '40817810000006468175',
  'account_valid_to': '2033-11-15T00:00:00',
  'client': '8-33140',
  'last_name': 'Ачентов',
  'first_name': 'Антон',
  'patronymic': 'Дмитриевич',
  'date_of_birth': '1960-08-04T00:00:00',
  'passport': 6403074379,
  'passport_valid_to': '2030-09-13T00:00:00'

In [4]:
transactions_df = pd.DataFrame.from_dict(content1, orient='index') # преобразование в dataframe

In [5]:
transactions_df.head(10) # посмотрим первые 10 строк датафрейма, чтобы убедиться, что json распознан правильно

,date,card,account,account_valid_to,client,last_name,first_name,patronymic,date_of_birth,passport,passport_valid_to,phone,oper_type,amount,oper_result,terminal,terminal_type,city,address
459270924,2020-05-01T00:00:29,59649132026167121328,40817810000001139973,2036-01-16T00:00:00,3-95179,Мисик,Сергей,Николаевич,1938-06-25T00:00:00,7076445954,2022-11-09T00:00:00,+79497481039,Пополнение,31576.60,Отказ,POS43792,POS,Славянск-на-Кубани,"Славянск-на-Кубани, ул. Клецкая, д. 86"
459270943,2020-05-01T00:02:46,54236863445669263724,40817810000006468175,2033-11-15T00:00:00,8-33140,Ачентов,Антон,Дмитриевич,1960-08-04T00:00:00,6403074379,2030-09-13T00:00:00,+79458324907,Пополнение,60041.10,Отказ,POS75616,POS,Пустошка,"Пустошка, ул. Цюрупинская, д. 78"
459270968,2020-05-01T00:05:14,45930894029725313464,40817810000009142118,2029-03-18T00:00:00,4-72043,Гуляков,Евгений,Константинович,1989-08-15T00:00:00,3354134791,2021-06-24T00:00:00,+79953939977,Снятие,18474.46,Успешно,POS28311,POS,Петровск-Забайкальский,"Петровск-Забайкальский, ул. Тагильская, д. 38"
459271057,2020-05-01T00:07:05,47550893546403715714,40817810000005559732,2028-04-01T00:00:00,1-85045,Гуляков,Кирилл,Петрович,1996-09-18T00:00:00,4783507057,2023-01-08T00:00:00,+79698158422,Пополнение,915.26,Успешно,ATM39555,ATM,Краснослободск,"Краснослободск, ул. Фогеля, д. 20"
459271064,2020-05-01T00:08:27,53036257457049008830,40817810000007495265,2037-10-26T00:00:00,5-77607,Мисик,Сергей,Олегович,1992-05-08T00:00:00,3195562178,2032-09-27T00:00:00,+79181011177,Оплата,75843.91,Отказ,ATM16730,ATM,Черкесск,"Черкесск, ул. Гинтовта, д. 56"
459271161,2020-05-01T00:09:16,42810999157069197448,40817810000003762026,2027-07-04T00:00:00,6-78748,Сноркин,Андрей,Антонович,1984-11-20T00:00:00,3602724796,2035-10-16T00:00:00,+79890926038,Оплата,52415.15,Успешно,POS64475,POS,Вичуга,"Вичуга, ул. Котляревского, д. 100"
459271162,2020-05-01T00:09:59,45080063716602427656,40817810000006914392,2032-06-08T00:00:00,6-40096,Ачентов,Владимир,Александрович,1943-06-19T00:00:00,6183431135,2040-12-09T00:00:00,+79943917322,Снятие,39265.25,Успешно,POS64475,POS,Вичуга,"Вичуга, ул. Котляревского, д. 100"
459271163,2020-05-01T00:11:37,59545483119724056521,40817810000001617265,2031-03-10T00:00:00,7-29452,Мисик,Андрей,Константинович,1950-09-10T00:00:00,9054168281,2031-01-17T00:00:00,+79652111522,Снятие,17880.89,Отказ,ATM90761,ATM,Морозовск,"Морозовск, ул. Васильковая, д. 13"
459271164,2020-05-01T00:13:49,42388100783866617513,40817810000001005464,2039-11-23T00:00:00,6-14693,Гладишкин,Александр,Михаилович,1996-01-13T00:00:00,7004798931,2035-05-26T00:00:00,+79239086361,Снятие,46074.67,Отказ,POS45875,POS,Воронеж,"Воронеж, ул. Раевского Николая, д. 22"
459271165,2020-05-01T00:14:48,43440125275047901246,40817810000005508931,2033-11-06T00:00:00,5-15302,Поджарый,Роман,Александрович,1951-07-24T00:00:00,7173036990,2036-08-05T00:00:00,+79343500689,Оплата,41054.44,Успешно,POS20393,POS,Тара,"Тара, ул. Уральская, д. 39"


## Часть вторая - Transform

### Обработка/подготовка к анализу

Проведем проверку уникальных/пустых значений в нашем датасете и очистим его от ненужных данных

In [7]:
# проверка на пустые значения
transactions_df.isna().any()

date                 False
card                 False
account              False
account_valid_to     False
client               False
last_name            False
first_name           False
patronymic           False
date_of_birth        False
passport             False
passport_valid_to    False
phone                False
oper_type            False
amount               False
oper_result          False
terminal             False
terminal_type        False
city                 False
address              False
dtype: bool

Пустых значений нет => меньше ступеней в обработке данных

In [11]:
# проверка на уникальные значения номеров кард, акков, id клиентов, сумма денег
columns_for_check = ['card', 'account', 'client', 'amount']
print('Уникальные значения в колонках:')
for column in columns_for_check:
    len_of_unique_values = len(pd.unique(transactions_df[column]))
    len_of_the_column = len(transactions_df[column])
    print(f"Длина колонки - {len_of_the_column}; количество уникальных значений - {len_of_unique_values}")
    print(f"{column} - {'все уникальные' if len_of_the_column - len_of_unique_values == 0 else 'есть одинаковые'}")
    print()

Уникальные значения в колонках:
Длина колонки - 4471; количество уникальных значений - 4192
card - есть одинаковые

Длина колонки - 4471; количество уникальных значений - 4192
account - есть одинаковые

Длина колонки - 4471; количество уникальных значений - 1100
client - есть одинаковые

Длина колонки - 4471; количество уникальных значений - 4470
amount - есть одинаковые



Есть повторяющиеся номера клиентов и аккаунтов/кард => дополнительные гипотезы для определения фрода

Нет сильно повторяющихся значений сумм транзакций => признаки фродв по суммам транзакций кажутся бессмысленными, но со счетов не сбрасываем пока


In [71]:
date_exp = pd.to_datetime(transactions_df.loc[:, 'date'].str.split('T').apply(lambda x: x[1]), format='%H:%M:%S').dt.time
date_exp

459270924    00:00:29
459270943    00:02:46
459270968    00:05:14
459271057    00:07:05
459271064    00:08:27
               ...   
459275623    21:33:12
459275624    21:34:46
459275625    21:36:07
459275626    21:37:48
459275627    21:39:29
Name: date, Length: 4471, dtype: object

In [74]:
# избавимся от ненужных данных и оставим следующие данные:
# - дата операции (разобьем отдельно на дату и время)
# - номер карты
# - номер акка
# - номер клиента
# - сумма средств в операции
# - город

df_for_analysis = transactions_df.loc[:, ['card', 'account', 'client', 'amount', 'city']]
transaction_dates = pd.to_datetime(transactions_df.loc[:, 'date'].str.split('T').apply(lambda x: x[0]), format='%Y-%m-%d')
df_for_analysis['date'] = transaction_dates
transaction_times = pd.to_datetime(transactions_df.loc[:, 'date'].str.split('T').apply(lambda x: x[1]), format='%H:%M:%S').dt.time
df_for_analysis['time'] = transaction_times
df_for_analysis

,card,account,client,amount,city,date,time
459270924,59649132026167121328,40817810000001139973,3-95179,31576.60,Славянск-на-Кубани,2020-05-01,00:00:29
459270943,54236863445669263724,40817810000006468175,8-33140,60041.10,Пустошка,2020-05-01,00:02:46
459270968,45930894029725313464,40817810000009142118,4-72043,18474.46,Петровск-Забайкальский,2020-05-01,00:05:14
459271057,47550893546403715714,40817810000005559732,1-85045,915.26,Краснослободск,2020-05-01,00:07:05
459271064,53036257457049008830,40817810000007495265,5-77607,75843.91,Черкесск,2020-05-01,00:08:27
...,...,...,...,...,...,...,...
459275623,48201610882535918830,40817810000009995390,1-56145,33456.18,Зима,2020-05-05,21:33:12
459275624,58685860072058751139,40817810000009995391,1-29405,60893.22,Выкса,2020-05-05,21:34:46
459275625,52002457754706265318,40817810000009995392,8-44184,17421.05,Ессентуки,2020-05-05,21:36:07
459275626,46860025272700909519,40817810000009995393,8-44184,27723.81,Ессентуки,2020-05-05,21:37:48
